<a href="https://colab.research.google.com/github/Jerry086/CS6140CarbonMapping/blob/main/Forest_Carbon_Mapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Forest Carbon Mapping

## Setting Up Enviornment

Mount Google Drive to the Colab VM if you want to save and load files from your Google Drive. Please note that the space in your Google Drive is limited, and you may need to upgrade your storage plan if you want to save large files.

In [1]:
# skip the code if dataset is uploaded or downloaded from Google Cloud Storage
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Authenticate Google Cloud Storage if data is stored and retrieved from it.

In [2]:
# authenticate google cloud
from google.colab import auth
auth.authenticate_user()

Authenticate and authorize access to Earth Engine.

In [3]:
# Import, authenticate and initialize the Earth Engine library.
import ee
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=ztq-_eH48dj7wEBoqsnrjKlVYllJinXyF2KMS1sMdow&tc=Nx8lt54O4PkUBTbRI2wsZTGxzjlklg9hr4944pefx6Q&cc=5O0cF3SVRfu4--quCuD5r2p_FMDmKGK0ruS8E_BVfjw

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1Adeu5BX-hGs3qVZOlr-9a0wLvOGt2vwdNsY3863D2k_PxIwssYqACttaUVU

Successfully saved authorization token.


## Install & import Python packages



Import Python packages used throughout this notebook.

In [4]:
# Folium setup.
import folium
print(folium.__version__)

0.14.0


In [5]:
# Tensorflow setup.
import tensorflow as tf
print(tf.__version__)

2.12.0


## Set global variables

In [6]:
# Specify names locations for outputs in Google Drive.
# Uncomment the code for Google Drive storage.
# FOLDER = 'dataset'

# Specify names locations for outputs in Google Cloud.
BUCKET = 'cs6140'
FOLDER = 'dataset'
TRAINING_BASE = 'training_patches'
EVAL_BASE = 'eval_patches'

# Specify feature bands to the model and the response variable.
MODIS_BANDS = ['EVI']
COPERNICUS_BANDS = ['discrete_classification', 'forest_type']
TERRA_BANDS = ['Percent_Tree_Cover', 'Percent_NonTree_Vegetation']
BANDS = MODIS_BANDS + COPERNICUS_BANDS + TERRA_BANDS
RESPONSE = 'annualNPP'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches (256x256 pixels images) expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]

# Columns for input features and response
COLUMNS = [
  # Configuration for parsing a fixed-length input feature.
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
# Label each column with feature name by dictionary
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

# Sizes of the training and evaluation datasets.
TRAIN_SIZE = 10000
EVAL_SIZE = 5000

# Specify model training parameters.
BATCH_SIZE = 16
EPOCHS = 2
BUFFER_SIZE = 500
OPTIMIZER = 'Adam'
LOSS = 'MeanSquaredError'
METRICS = ['RootMeanSquaredError']

# Visualizing Images and Image Bands

This research focuses on Forest Carbon Mapping in North America area

In [7]:
# Define the region of interest (ROI)
north_america_polygon = ee.Geometry.Polygon([
    [-168, 65],  # Northwest corner (top-left)
    [-168, 10],  # Southwest corner (bottom-left)
    [-52, 10],   # Southeast corner (bottom-right)
    [-52, 65],   # Northeast corner (top-right)
])

In [8]:
def reduce_collection(filename):
  """The filter function.
  Filter an image collection to the the year of 2018, region of North America
  Reduce temporal data by mean
  Args:
    filename: dataset filename of the collection.
  Returns:
    A representative image.
  """
  collection = ee.ImageCollection(filename)
  filtered = collection.filterDate('2018-01-01', '2018-12-31')
  reduced = filtered.mean().clip(north_america_polygon)
  return reduced

def normalize_band(image, band):
  """The scaling function.
  Scaling an image band within the range [0, 1] by min-max scaler
  Args:
    image: the image.
    band: the band to be scaled.
  Returns:
    A scaled image band.
  """
  image = image.select(band)
  stats = image.reduceRegion(
      reducer=ee.Reducer.minMax(),
      geometry=north_america_polygon,
      scale=250,
      bestEffort=True
  ).getInfo()
  min, max = stats[band + '_min'], stats[band + '_max']
  normalized = image.unitScale(min, max)
  return normalized

Visualize 'EVI' band from [MOD13Q1.061 Terra Vegetation Indices 16-Day Global 250m](https://developers.google.com/earth-engine/datasets/catalog/MODIS_061_MOD13Q1#bands).

In [9]:
modis_image = reduce_collection("MODIS/061/MOD13Q1")
EVI = normalize_band(modis_image, 'EVI')

# Use folium to visualize the image
mapid = EVI.getMapId({'bands': ['EVI'], 'min': 0, 'max': 1, 'palette': [
    'ffffff', 'ce7e45', 'df923d', 'f1b555', 'fcd163', '99b718', '74a901',
    '66a000', '529400', '3e8601', '207401', '056201', '004c00', '023b01',
    '012e01', '011d01', '011301'
  ]})
# Center in Vancouver
map = folium.Map(location=[49.2827, -123.1207])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='EVI',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Visualize 'Discrete_classification' and 'forest_type' bands from [Copernicus Global Land Cover Layers: CGLS-LC100 Collection 3](https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_Landcover_100m_Proba-V-C3_Global#bands).

In [35]:
copernicus_image = reduce_collection("COPERNICUS/Landcover/100m/Proba-V-C3/Global")
discrete_classification = normalize_band(copernicus_image, 'discrete_classification')
forest_type = normalize_band(copernicus_image, 'forest_type')

# Use folium to visualize the image
mapid = discrete_classification.getMapId({'bands': ['discrete_classification'], 'min': 0, 'max': 1, 'palette': [
    '282828', '666000', '009900', '70663e', 'a0dc00', '929900'
  ]})
map = folium.Map(location=[49.2827, -123.1207])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='discrete_classification',
  ).add_to(map)

mapid = forest_type.getMapId({'bands': ['forest_type'], 'min': 0, 'max': 1, 'palette': [
    '282828', '666000', '009900', '70663e', 'a0dc00', '929900'
  ]})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='forest_type',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Visualize 'Percent_Tree_Cover' and 'Percent_NonTree_Vegetation' from [MOD44B.006 Terra Vegetation Continuous Fields Yearly Global 250m](https://developers.google.com/earth-engine/datasets/catalog/MODIS_006_MOD44B#bands).

In [11]:
terra_image = reduce_collection("MODIS/006/MOD44B")
Percent_Tree_Cover = normalize_band(terra_image, 'Percent_Tree_Cover')
Percent_NonTree_Vegetation = normalize_band(terra_image, 'Percent_NonTree_Vegetation')

# Use folium to visualize the image
mapid = Percent_Tree_Cover.getMapId({'bands': ['Percent_Tree_Cover'], 'min': 0, 'max': 1, 'palette': ['bbe029', '0a9501', '074b03']})
map = folium.Map(location=[49.2827, -123.1207])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Percent_Tree_Cover',
  ).add_to(map)

mapid = Percent_NonTree_Vegetation.getMapId({'bands': ['Percent_NonTree_Vegetation'], 'min': 0, 'max': 1, 'palette': ['bbe029', '0a9501', '074b03']})
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='Percent_NonTree_Vegetation',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Response: the amount of carbon captured by plants in an ecosystem, after accounting for losses due to respiration, from [MODIS Net Primary Production CONUS](https://developers.google.com/earth-engine/datasets/catalog/UMT_NTSG_v2_MODIS_NPP#bands).

In [12]:
npp_image = reduce_collection('UMT/NTSG/v2/MODIS/NPP')
annualNPP = normalize_band(npp_image, 'annualNPP')

mapid = annualNPP.getMapId({'bands': ['annualNPP'], 'min': 0, 'max': 1, 'palette': ['bbe029', '0a9501', '074b03']})
map = folium.Map(location=[38., -122.5])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='npp',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

Stack the 2D images (Feature Bands and Response) to create a single image from which samples can be taken. Convert the image into an array image in which each pixel stores 256x256 patches of pixels for each band. To export training patches, convert a multi-band image to an array image using neighborhoodToArray(), then sample the image at points.


In [13]:
featureStack = ee.Image.cat([
  EVI.select('EVI'),
  discrete_classification.select('discrete_classification'),
  forest_type.select('forest_type'),
  Percent_Tree_Cover.select('Percent_Tree_Cover'),
  Percent_NonTree_Vegetation.select('Percent_NonTree_Vegetation'),
  annualNPP.select('annualNPP')
]).float()

list = ee.List.repeat(1, KERNEL_SIZE)
lists = ee.List.repeat(list, KERNEL_SIZE)
kernel = ee.Kernel.fixed(KERNEL_SIZE, KERNEL_SIZE, lists)

arrays = featureStack.neighborhoodToArray(kernel)

Utilize geometries from various U.S. states to extract samples from the stack at strategic points, focusing on areas with the highest forest coverage. Specifically, these are imported polygons in which to take the 256x256 samples. Display the sampling polygons on a map, red for training polygons, blue for evaluation.

In [14]:
usa = ee.FeatureCollection('projects/ee-jariwaladh0/assets/usa')
trainingPolys = usa.filter(ee.Filter.inList('shapeName', ['Maine', 'Alabama', 'South Carolina', 'New Hampshire','West Virginia','Vermont']));
evalPolys = usa.filter(ee.Filter.inList('shapeName', ['Georgia', 'Virginia']));

polyImage = ee.Image(0).byte().paint(trainingPolys, 1).paint(evalPolys, 2)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})
map = folium.Map(location=[38., -100.], zoom_start=5)
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='training polygons',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

# Sampling

Take a sample from each polygon and merge the results into a single export.  The key step is sampling the array image at points, to get all the pixels in a 256x256 neighborhood at each point. Export a single TFRecord file that contains patches of pixel values in each record to build the training and testing data for the model. Since each record potentially contains a lot of data (especially with big patches or many input bands), some manual sharding of the computation is necessary to avoid the `computed value too large` error.  Specifically, the following code takes multiple (smaller) samples within each geometry, merging the results to get a single export.

In [17]:
# Convert the feature collections to lists for iteration.
trainingPolysList = trainingPolys.toList(trainingPolys.size())
evalPolysList = evalPolys.toList(evalPolys.size())

n = 200 # Number of shards in each polygon, reduced to 200 samples
N = 2000 # Total sample size in each polygon.

# Export all the training data (in many pieces), with one task per geometry.
for g in range(trainingPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  # randomly choose one sample in each iteration
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(trainingPolysList.get(g)).geometry(),
      scale = 250,
      numPixels = N / n, # Size of the shard.
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)

  desc = TRAINING_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc,
    # Google Cloud Storage export, for Google Drive export, refer to toDrive()
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()

# Export all the evaluation data.
for g in range(evalPolys.size().getInfo()):
  geomSample = ee.FeatureCollection([])
  for i in range(n):
    sample = arrays.sample(
      region = ee.Feature(evalPolysList.get(g)).geometry(),
      scale = 250,
      numPixels = N / n,
      seed = i,
      tileScale = 8
    )
    geomSample = geomSample.merge(sample)

  desc = EVAL_BASE + '_g' + str(g)
  task = ee.batch.Export.table.toCloudStorage(
    collection = geomSample,
    description = desc,
    # Google Cloud Storage export, for Google Drive export, refer to toDrive()
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + desc,
    fileFormat = 'TFRecord',
    selectors = BANDS + [RESPONSE]
  )
  task.start()

# Training data

Load the data exported from Earth Engine into a `tf.data.Dataset`.

In [18]:
def parse_tfrecord(example_proto):
  """The parsing function.
  Read a serialized example into the structure defined by FEATURES_DICT.
  Args:
    example_proto: a serialized Example.
  Returns:
    A dictionary of tensors, keyed by feature name.
  """
  return tf.io.parse_single_example(example_proto, FEATURES_DICT)


def to_tuple(inputs):
  """Function to convert a dictionary of tensors to a tuple of (inputs, outputs).
  Turn the tensors returned by parse_tfrecord into a stack in HWC shape.
  Args:
    inputs: A dictionary of tensors, keyed by feature name.
  Returns:
    A tuple of (inputs, outputs).
  """
  inputsList = [inputs.get(key) for key in FEATURES]
  stacked = tf.stack(inputsList, axis=0)
  # Convert from CHW to HWC
  stacked = tf.transpose(stacked, [1, 2, 0])
  return stacked[:,:,:len(BANDS)], stacked[:,:,len(BANDS):]


def get_dataset(pattern):
  """Function to read, parse and format to tuple a set of input tfrecord files.
  Get all the files matching the pattern, parse and convert to tuple.
  Args:
    pattern: A file pattern to match in a Cloud Storage bucket.
  Returns:
    A tf.data.Dataset
  """
  glob = tf.io.gfile.glob(pattern)
  dataset = tf.data.TFRecordDataset(glob, compression_type='GZIP')
  dataset = dataset.map(parse_tfrecord, num_parallel_calls=5)
  dataset = dataset.map(to_tuple, num_parallel_calls=5)
  return dataset

Use the helpers to read in the training dataset.  Print the first record to check.

In [19]:
def get_training_dataset():
	"""Get the preprocessed training dataset
  Returns:
    A tf.data.Dataset of training data.
  """
	# directory for Google Cloud Storage, to read from Google Drive, modify 'glob' accordingly
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + TRAINING_BASE + '*'
	dataset = get_dataset(glob)
	# shuffle in n iterations, random pick one element from buffer in each iteration
	# batch in size BATCH_SIZE
	# repeat when all element are comsumed
	dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).repeat()
	return dataset

training = get_training_dataset()
print(iter(training.take(1)).next())

(<tf.Tensor: shape=(16, 256, 256, 5), dtype=float32, numpy=
array([[[[0.6880596 , 0.57      , 0.8       , 0.8452381 , 0.21428572],
         [0.69240767, 0.57      , 0.8       , 0.78571427, 0.25510204],
         [0.66608185, 0.62      , 0.        , 0.71428573, 0.31632653],
         ...,
         [0.675602  , 0.2       , 0.        , 0.20238096, 0.6938776 ],
         [0.66550046, 0.2       , 0.        , 0.1904762 , 0.71428573],
         [0.67202246, 0.2       , 0.        , 0.17857143, 0.7244898 ]],

        [[0.7142337 , 0.57      , 0.8       , 0.85714287, 0.1734694 ],
         [0.69921786, 0.57      , 0.8       , 0.8333333 , 0.20408164],
         [0.6777204 , 0.57      , 0.8       , 0.79761904, 0.23469388],
         ...,
         [0.69584054, 0.63      , 0.        , 0.23809524, 0.68367344],
         [0.693439  , 0.2       , 0.        , 0.22619048, 0.6938776 ],
         [0.71218103, 0.2       , 0.        , 0.15476191, 0.6938776 ]],

        [[0.70216036, 0.57      , 0.8       , 0.88095236

# Evaluation data

Now do the same thing to get an evaluation dataset.  Note that unlike the training dataset, the evaluation dataset has a batch size of 1, is not repeated and is not shuffled.

In [20]:
def get_eval_dataset():
	"""Get the preprocessed evaluation dataset
  Returns:
    A tf.data.Dataset of evaluation data.
  """
	# directory for Google Cloud Storage, to read from Google Drive, modify 'glob' accordingly
	glob = 'gs://' + BUCKET + '/' + FOLDER + '/' + EVAL_BASE + '*'
	dataset = get_dataset(glob)
	dataset = dataset.batch(1).repeat()
	return dataset

evaluation = get_eval_dataset()
evaluation

<_RepeatDataset element_spec=(TensorSpec(shape=(None, 256, 256, 5), dtype=tf.float32, name=None), TensorSpec(shape=(None, 256, 256, 1), dtype=tf.float32, name=None))>

# Model

Keras implementation of the U-Net model.  The U-Net model takes 256x256 pixel patches as input and a continuous output per-pixel.  We will use mean squared error loss on the sigmoidal output since we are treating this as a regression problem.

In [22]:
from keras.models import *
from keras.layers import *
from keras import metrics
from keras import optimizers
from keras import losses

def conv_block(input_tensor, num_filters):
	encoder = Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = BatchNormalization()(encoder)
	encoder = Activation('relu')(encoder)
	encoder = Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = BatchNormalization()(encoder)
	encoder = Activation('relu')(encoder)
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = concatenate([concat_tensor, decoder], axis=-1)
	decoder = BatchNormalization()(decoder)
	decoder = Activation('relu')(decoder)
	decoder = Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = BatchNormalization()(decoder)
	decoder = Activation('relu')(decoder)
	decoder = Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = BatchNormalization()(decoder)
	decoder = Activation('relu')(decoder)
	return decoder

def get_model():
	inputs = Input(shape=[None, None, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = Conv2D(1, (1, 1), activation='sigmoid')(decoder0)

	model = Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER),
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

# Training the model

Here we're going to train for 2 epochs, where the loss reaches a plateau experimentally

In [25]:
m = get_model()

m.fit(
    x=training,
    epochs=EPOCHS,
    steps_per_epoch=int(TRAIN_SIZE / BATCH_SIZE),
    validation_data=evaluation,
    validation_steps=EVAL_SIZE)

# Save the weights after training
m.save_weights('drive/My Drive/dataset/params/normalized')

Epoch 1/2
625/625 [==============================] - 1105s 2s/step - loss: 0.0125 - root_mean_squared_error: 0.1117 - val_loss: 0.0882 - val_root_mean_squared_error: 0.2969
Epoch 2/2
625/625 [==============================] - 1003s 2s/step - loss: 0.0120 - root_mean_squared_error: 0.1094 - val_loss: 0.0304 - val_root_mean_squared_error: 0.1744


In [26]:
# Display the model's architecture
m.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, None, None,  0           []                               
                                 5)]                                                              
                                                                                                  
 conv2d_23 (Conv2D)             (None, None, None,   1472        ['input_2[0][0]']                
                                32)                                                               
                                                                                                  
 batch_normalization_27 (BatchN  (None, None, None,   128        ['conv2d_23[0][0]']              
 ormalization)                  32)                                                         

# Retriving the model's weights

To avoid retrain the model, we can retrive its weights

In [ ]:
m = get_model()
# Restore the weights
m.load_weights('drive/My Drive/dataset/params/normalized')

# Evaluate the model
loss, acc = m.evaluate(x=evaluation, verbose=2, steps=1000)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))


1000/1000 - 769s - loss: 0.0392 - root_mean_squared_error: 0.1980 - 769s/epoch - 769ms/step
Restored model, accuracy: 19.80%


# Prediction

The prediction pipeline is:

1.  Export imagery on which to do predictions from Earth Engine in TFRecord format to a Cloud Storage bucket.
2.  Use the trained model to make the predictions.
3.  Write the predictions to a TFRecord file in a Cloud Storage.
4.  Upload the predictions TFRecord file to Earth Engine.


In [29]:
def doExport(inputs, out_image_base, kernel_buffer, region):
  """Run the image export task.  Block until complete.
  """
  task = ee.batch.Export.image.toCloudStorage(
    image = inputs.select(BANDS),
    description = out_image_base,
    # Google Cloud Storage export, for Google Drive export, refer to toDrive()
    bucket = BUCKET,
    fileNamePrefix = FOLDER + '/' + out_image_base,
    region = region.getInfo()['coordinates'],
    scale = 250,
    fileFormat = 'TFRecord',
    maxPixels = 1e10,
    formatOptions = {
      'patchDimensions': KERNEL_SHAPE,
      'kernelSize': kernel_buffer,
      'compressed': True,
      'maxFileSize': 104857600
    }
  )
  task.start()

  # Block until the task completes.
  print('Running image export to Google Cloud/Drive...')
  import time
  while task.active():
    time.sleep(30)

  # Error condition
  if task.status()['state'] != 'COMPLETED':
    print('Error with image export.')
  else:
    print('Image export completed.')

Now there's all the code needed to run the prediction pipeline, all that remains is to specify the output region in which to do the prediction, the names of the output files, where to put them, and the shape of the outputs.  In terms of the shape, the model is trained on 256x256 patches, but can work (in theory) on any patch that's big enough with even dimensions.  Because of tile boundary artifacts, give the model slightly larger patches for prediction, then clip out the middle 256x256 patch.  This is controlled with a kernel buffer, half the size of which will extend beyond the kernel buffer.  For example, specifying a 128x128 kernel will append 64 pixels on each side of the patch, to ensure that the pixels in the output are taken from inputs completely covered by the kernel.

In [30]:
# prepare for prediction
m = get_model()
# Restore the weights that have been uploaded to Google Cloud Storage
m.load_weights('gs://cs6140/params/normalized')

In [31]:
# input features
inputs = ee.Image.cat([
  EVI.select('EVI'),
  discrete_classification.select('discrete_classification'),
  forest_type.select('forest_type'),
  Percent_Tree_Cover.select('Percent_Tree_Cover'),
  Percent_NonTree_Vegetation.select('Percent_NonTree_Vegetation')
]).float()

##Prediction for the US region

In [32]:
# Output assets folder:
user_folder = 'users/weizhon'

# Base file name to use for TFRecord files and assets.
us_image_base = 'region_US_'
# Half this will extend on the sides of each patch.
us_kernel_buffer = [128, 128]

# US
us_region = ee.Geometry.Polygon(
        [[[-124.81863747950683,48.366802253368554],
          [-124.64285622950683,39.39753877975156],
          [-117.78738747950683,32.578718148918284],
          [-115.32644997950683,32.578718148918284],
          [-110.40457497950683,31.310898910711277],
          [-106.53738747950683,31.535899874858018],
          [-101.43973122950685,29.338567087853676],
          [-97.22098122950685,25.913049047273233],
          [-93.79324685450685,28.56952616533715],
          [-84.03738747950685,29.415156022764283],
          [-82.54324685450685,24.801153626796097],
          [-79.20340310450685,25.278931513895785],
          [-81.13699685450685,31.01005996380629],
          [-75.86355935450685,35.49149813673187],
          [-66.89871560450685,44.554824331783905],
          [-67.77762185450685,47.125807684916175],
          [-69.09598122950685,47.42396746903084],
          [-71.38113747950685,45.11575579190003],
          [-74.80887185450685,45.05369971520998],
          [-76.56668435450685,43.861827567393036],
          [-79.37918435450685,43.67141365534179],
          [-78.85184060450685,42.774805662109834],
          [-82.63113747950685,41.79953870387609],
          [-83.07059060450685,45.609777722861956],
          [-87.90457497950685,48.07401028927917],
          [-94.84793435450685,48.947367346362974],
          [-99.33035622950685,48.947367346362974],
          [-111.37137185450683,48.947367346362974],
          [-123.23660622950683,48.947367346362974],
          [-124.81863747950683,48.366802253368554]]], None, False)

In [33]:
us_geojson = us_region.getInfo()
map = folium.Map(location=[35.355,-98], zoom_start=4)
folium.GeoJson(us_geojson, name='us').add_to(map)
map

In [ ]:
# Run the export.
doExport(inputs, us_image_base, us_kernel_buffer, us_region)

Running image export to Google Drive...
Image export completed.


In [34]:
def doPrediction(out_image_base, user_folder, kernel_buffer, region):
  """Perform inference on exported imagery, upload to Earth Engine.
  """

  print('Looking for TFRecord files...')

  # Get a list of all the files in the output bucket.
  # gdrive
  # filesList = !ls -1 'drive/My Drive/'{FOLDER}
  filesList = !gsutil ls 'gs://'{BUCKET}'/'{FOLDER}
  # Get only the files generated by the image export.
  exportFilesList = [s for s in filesList if out_image_base in s]

  # Get the list of image files and the JSON mixer file.
  imageFilesList = []
  jsonFile = None
  for f in exportFilesList:
    if f.endswith('.tfrecord.gz'):
      imageFilesList.append(f)
    elif f.endswith('.json'):
      jsonFile = f

  # Make sure the files are in the right order.
  imageFilesList.sort()
  # gdrive
  # for i, file in enumerate(imageFilesList):
  #   imageFilesList[i] = 'drive/My Drive/' + FOLDER + '/' + file
  from pprint import pprint
  pprint(imageFilesList)
  print(jsonFile)

  import json
  # Load the contents of the mixer file to a JSON object.
  # gdrive
  # jsonText = !cat 'drive/My Drive/'{FOLDER}'/'{jsonFile}
  jsonText = !gsutil cat {jsonFile}
  # Get a single string w/ newlines from the IPython.utils.text.SList
  mixer = json.loads(jsonText.nlstr)
  pprint(mixer)
  patches = mixer['totalPatches']

  # Get set up for prediction.
  x_buffer = int(kernel_buffer[0] / 2)
  y_buffer = int(kernel_buffer[1] / 2)

  buffered_shape = [
      KERNEL_SHAPE[0] + kernel_buffer[0],
      KERNEL_SHAPE[1] + kernel_buffer[1]]

  imageColumns = [
    tf.io.FixedLenFeature(shape=buffered_shape, dtype=tf.float32)
      for k in BANDS
  ]

  imageFeaturesDict = dict(zip(BANDS, imageColumns))

  def parse_image(example_proto):
    return tf.io.parse_single_example(example_proto, imageFeaturesDict)

  def toTupleImage(inputs):
    inputsList = [inputs.get(key) for key in BANDS]
    stacked = tf.stack(inputsList, axis=0)
    stacked = tf.transpose(stacked, [1, 2, 0])
    return stacked

   # Create a dataset from the TFRecord file(s) in Google Drive.
  imageDataset = tf.data.TFRecordDataset(imageFilesList, compression_type='GZIP')
  imageDataset = imageDataset.map(parse_image, num_parallel_calls=5)
  imageDataset = imageDataset.map(toTupleImage).batch(1)

  # Perform inference.
  print('Running predictions...')
  predictions = m.predict(imageDataset, steps=patches, verbose=1)
  # print(predictions[0])

  print('Writing predictions...')
  # gDrive
  # out_image_file = 'drive/My Drive/' + FOLDER + '/' + out_image_base + '.TFRecord'
  out_image_file = 'gs://' + BUCKET + '/' + FOLDER + '/' + out_image_base + '.TFRecord'
  writer = tf.io.TFRecordWriter(out_image_file)
  patches = 0
  for predictionPatch in predictions:
    print('Writing patch ' + str(patches) + '...')
    predictionPatch = predictionPatch[
        x_buffer:x_buffer+KERNEL_SIZE, y_buffer:y_buffer+KERNEL_SIZE]

    # Create an example.
    example = tf.train.Example(
      features=tf.train.Features(
        feature={
          'impervious': tf.train.Feature(
              float_list=tf.train.FloatList(
                  value=predictionPatch.flatten()))
        }
      )
    )
    # Write the example.
    writer.write(example.SerializeToString())
    patches += 1

  writer.close()

  # Start the upload.
  out_image_asset = user_folder + '/' + out_image_base
  !earthengine upload image --asset_id={out_image_asset} {out_image_file} {jsonFile}

In [ ]:
# Run the prediction.
doPrediction(us_image_base, user_folder, us_kernel_buffer, us_region)

Looking for TFRecord files...
['gs://cs6140/dataset/region_US_00000.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00001.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00002.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00003.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00004.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00005.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00006.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00007.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00008.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00009.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00010.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00011.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00012.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00013.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00014.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00015.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00016.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00017.tfrecord.gz',
 'gs://cs6140/dataset/region_US_00

##Prediction for Canada Region

You may need to request a high-RAM for the prediction of Canada

In [36]:
# Base file name to use for TFRecord files and assets.
ca_image_base = 'region_CA_'
# Half this will extend on the sides of each patch.
ca_kernel_buffer = [128, 128]

# Canada polygon
ca_region = ee.Geometry.Polygon(
        [[[-141.18426082833156,69.74644944713596],
          [-141.00847957833156,60.017918210946156],
          [-123.78191707833156,47.926998166924115],
          [-94.42644832833156,48.62889591883798],
          [-85.46160457833156,47.45370326106549],
          [-81.59441707833156,43.51228551542987],
          [-83.17644832833156,41.4381749493045],
          [-74.38738582833156,44.89827737618559],
          [-70.52019832833156,45.270613757097955],
          [-68.93816707833156,47.45370326106549],
          [-66.65301082833156,43.76670940415023],
          [-62.78582332833155,42.35405729936129],
          [-49.60222957833155,47.69088762394784],
          [-63.84051082833155,60.280398658664566],
          [-77.90301082833156,63.03228611232247],
          [-80.18816707833156,51.5596378647505],
          [-81.77019832833156,55.22556037192514],
          [-93.89910457833156,59.127747820685045],
          [-84.93426082833156,66.26441257319317],
          [-80.01238582833156,67.23571238025826],
          [-81.59441707833156,69.68551069309922],
          [-95.30535457833156,69.3781744685675],
          [-113.58660457833156,68.16929753552228],
          [-128.00066707833156,70.46417725306067],
          [-141.18426082833156,69.74644944713596]]], None, False)

In [37]:
ca_geojson = ca_region.getInfo()
map = folium.Map(location=[65.355,-98], zoom_start=4)
folium.GeoJson(ca_geojson, name='us').add_to(map)
map

In [ ]:
# Run the export.
doExport(inputs, ca_image_base, ca_kernel_buffer, ca_region)

Running image export to Google Cloud/Drive...
Image export completed.


In [ ]:
# Run the prediction.
doPrediction(ca_image_base, user_folder, ca_kernel_buffer, ca_region)

Streaming output truncated to the last 5000 lines.
Writing patch 2951...
Writing patch 2952...
Writing patch 2953...
Writing patch 2954...
Writing patch 2955...
Writing patch 2956...
Writing patch 2957...
Writing patch 2958...
Writing patch 2959...
Writing patch 2960...
Writing patch 2961...
Writing patch 2962...
Writing patch 2963...
Writing patch 2964...
Writing patch 2965...
Writing patch 2966...
Writing patch 2967...
Writing patch 2968...
Writing patch 2969...
Writing patch 2970...
Writing patch 2971...
Writing patch 2972...
Writing patch 2973...
Writing patch 2974...
Writing patch 2975...
Writing patch 2976...
Writing patch 2977...
Writing patch 2978...
Writing patch 2979...
Writing patch 2980...
Writing patch 2981...
Writing patch 2982...
Writing patch 2983...
Writing patch 2984...
Writing patch 2985...
Writing patch 2986...
Writing patch 2987...
Writing patch 2988...
Writing patch 2989...
Writing patch 2990...
Writing patch 2991...
Writing patch 2992...
Writing patch 2993...
Wri

# Display the output

Once the data has been exported, the model has made predictions and the predictions have been written to a file, and the image imported to Earth Engine, it's possible to display the resultant Earth Engine asset.  Here, display the impervious area predictions over the US and Canada.

In [38]:
out_image = ee.Image(user_folder + '/' + us_image_base)
out_image = out_image.clip(usa)
mapid = out_image.getMapId({'min': 0, 'max': 1, 'palette': ['bbe029', '0a9501', '074b03']})
map = folium.Map(location=[35,-98])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='predicted US',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

In [39]:
countries = ee.FeatureCollection("FAO/GAUL/2015/level0")
canada = countries.filter(ee.Filter.eq('ADM0_CODE', 46))
out_image = ee.Image(user_folder + '/' + ca_image_base)
out_image = out_image.clip(canada)
mapid = out_image.getMapId({'min': 0, 'max': 1, 'palette': ['bbe029', '0a9501', '074b03']})
map = folium.Map(location=[49.2827, -123.1207])
folium.TileLayer(
    tiles=mapid['tile_fetcher'].url_format,
    attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    overlay=True,
    name='predicted Canada',
  ).add_to(map)
map.add_child(folium.LayerControl())
map

# Reference

We acknowledge the use of [TensorFlow example workflows](https://developers.google.com/earth-engine/guides/tf_examples) for inspiration and guidance in this notebook.